In [1]:
import pandas as pd
import os

# Load Path 
data_dir = "/Users/zhangminzhengxiong/Desktop/Current Projects/Equity Analysis/Data/SecFiles/13F/01dec2024-28feb2025_form13f/"


In [ ]:

# Read 13F files 
submission_df = pd.read_csv(os.path.join(data_dir, "SUBMISSION.tsv"), sep="\t", dtype=str)
coverpage_df = pd.read_csv(os.path.join(data_dir, "COVERPAGE.tsv"), sep="\t", dtype=str)
infotable_df = pd.read_csv(os.path.join(data_dir, "INFOTABLE.tsv"), sep="\t", dtype=str)
summary_df = pd.read_csv(os.path.join(data_dir, "SUMMARYPAGE.tsv"), sep="\t", dtype=str)


In [19]:
# Examples 
print("📄 SUBMISSION:")
print(submission_df.head())

print("\n📄 COVERPAGE:")
print(coverpage_df[['ACCESSION_NUMBER', 'FILINGMANAGER_NAME']].head())

print("\n📄 INFOTABLE:")
print(infotable_df[['ACCESSION_NUMBER', 'NAMEOFISSUER', 'CUSIP', 'VALUE', 'SSHPRNAMT']].head())

print("\n📄 SUMMARYPAGE:")
print(summary_df.head())

📄 SUBMISSION:
       ACCESSION_NUMBER  FILING_DATE SUBMISSIONTYPE         CIK PERIODOFREPORT
0  0001088875-25-000002  31-JAN-2025         13F-HR  0001088875    31-DEC-2024
1  0001957363-25-000001  31-JAN-2025         13F-HR  0001957363    31-DEC-2024
2  0001088875-25-000004  31-JAN-2025         13F-NT  0001088876    31-DEC-2024
3  0001706836-25-000002  31-JAN-2025         13F-HR  0001706836    31-DEC-2024
4  0000922423-25-000009  31-JAN-2025         13F-HR  0001407024    31-DEC-2024

📄 COVERPAGE:
       ACCESSION_NUMBER                        FILINGMANAGER_NAME
0  0002031775-25-000001       BridgePort Financial Solutions, LLC
1  0001628280-25-008999        Provida Pension Fund Administrator
2  0002058285-25-000002          Park Square Financial Group, LLC
3  0001667731-25-000316  Total Wealth Planning & Management, Inc.
4  0001803156-25-000001                Menard Financial Group LLC

📄 INFOTABLE:
       ACCESSION_NUMBER    NAMEOFISSUER      CUSIP    VALUE SSHPRNAMT
0  0002031775-25-0

In [15]:
# Merge Data 
meta_df = pd.merge(submission_df[["ACCESSION_NUMBER", "PERIODOFREPORT", "SUBMISSIONTYPE", "CIK"]],
                   coverpage_df[["ACCESSION_NUMBER", "FILINGMANAGER_NAME"]],
                   on="ACCESSION_NUMBER", how="inner")

combined_df = pd.merge(infotable_df, meta_df, on="ACCESSION_NUMBER", how="inner")

# Rename 
final_df = combined_df[[
    "FILINGMANAGER_NAME",
    "NAMEOFISSUER",
    "CUSIP",
    "PERIODOFREPORT",
    "SSHPRNAMT",
    "VALUE",
    "SUBMISSIONTYPE",
    "CIK",
    "INVESTMENTDISCRETION",
    "OTHERMANAGER",
    "ACCESSION_NUMBER"
]].rename(columns={
    "PERIODOFREPORT": "DATE",
    "SSHPRNAMT": "NUMBER_OF_SHARES",
    "VALUE": "NOTIONAL_VALUE", 
})



In [16]:
final_df.columns

Index(['FILINGMANAGER_NAME', 'NAMEOFISSUER', 'CUSIP', 'DATE',
       'NUMBER_OF_SHARES', 'NOTIONAL_VALUE', 'SUBMISSIONTYPE', 'CIK',
       'INVESTMENTDISCRETION', 'OTHERMANAGER', 'ACCESSION_NUMBER'],
      dtype='object')

In [17]:
# dtype transform 
final_df["NUMBER_OF_SHARES"] = pd.to_numeric(final_df["NUMBER_OF_SHARES"], errors='coerce')
final_df["NOTIONAL_VALUE"] = pd.to_numeric(final_df["NOTIONAL_VALUE"], errors='coerce')

# show 
print(final_df.head())

                    FILINGMANAGER_NAME    NAMEOFISSUER      CUSIP  \
0  BridgePort Financial Solutions, LLC           3M CO  88579Y101   
1  BridgePort Financial Solutions, LLC     ABBOTT LABS  002824100   
2  BridgePort Financial Solutions, LLC    ALPHABET INC  02079K305   
3  BridgePort Financial Solutions, LLC    ALPHABET INC  02079K107   
4  BridgePort Financial Solutions, LLC  AMAZON COM INC  023135106   

          DATE  NUMBER_OF_SHARES  NOTIONAL_VALUE SUBMISSIONTYPE         CIK  \
0  31-DEC-2024              4073          613102         13F-HR  0002031775   
1  31-DEC-2024              2171          295022         13F-HR  0002031775   
2  31-DEC-2024              7837         1320550         13F-HR  0002031775   
3  31-DEC-2024              3896          663126         13F-HR  0002031775   
4  31-DEC-2024              9660         2016335         13F-HR  0002031775   

  INVESTMENTDISCRETION OTHERMANAGER      ACCESSION_NUMBER  
0                 SOLE            0  0002031775-25

In [ ]:
📄 Documentation: 13F Institutional Holdings Dataset

This dataset is derived from the SEC Form 13F filings and contains detailed information on institutional investment holdings reported quarterly. Each row represents a single holding by an institutional filer for a specific security as of the end of the reporting period.

✅ Column Descriptions

Column Name	Type	Description
FILINGMANAGER_NAME	str	The name of the institutional investment manager who submitted the 13F filing.
NAMEOFISSUER	str	The name of the issuer of the security (i.e., the company whose stock is being reported).
CUSIP	str	The 9-digit CUSIP identifier that uniquely identifies the security.
DATE	str	The end date of the reporting period (e.g., 31-DEC-2024). This represents the quarter-end date of the filing.
NUMBER_OF_SHARES	int	The number of shares held by the filing manager for the reported security.
NOTIONAL_VALUE	float	The market value of the holding in USD. As of 2023, this value is reported in whole dollars (not in thousands).
SUBMISSIONTYPE	str	The type of SEC submission:
- 13F-HR: Initial quarterly holdings report
- 13F-HR/A: Amended holdings report
- 13F-NT: Notice report (no holdings disclosed).
CIK	str	Central Index Key – a unique identifier assigned by the SEC to the institutional filer. Useful for tracking a manager across filings.
INVESTMENTDISCRETION	str	Indicates who holds discretion over investment decisions:
- SOLE: Sole discretion
- SHARED: Shared discretion
- NONE: No discretion (e.g., managed by another entity).
OTHERMANAGER	str	The sequence number of other managers involved in managing the same holding. 0 or blank means no other manager is involved.
ACCESSION_NUMBER	str	Unique identifier assigned by the SEC for each filing. Can be used to locate the original filing on the SEC EDGAR website.

📌 Suggested Use Cases
Use FILINGMANAGER_NAME and DATE to track quarterly holdings by institution.

Use NAMEOFISSUER and DATE to identify the largest shareholders of a particular security.

Use CIK to track filings across years or merge with other SEC datasets.

Combine with external data (e.g., shares outstanding) to estimate ownership percentage.

Use INVESTMENTDISCRETION to distinguish between active and passive positions.

📎 Data Source
This dataset is constructed from the SEC’s Form 13F Quarterly Data Files, based on tab-separated values (TSV) provided in structured format.